In [1]:
import copy

# 外皮情報LV3からLV4へのコンバート

## I. 関数

### 1. 間仕切り床

#### 1) 床の仕様

| 番号 | 材料 | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> | 厚さ, m |
|---|---|---|---|---|
| 1 | 合板 | 0.16 | 720.0 | 0.024 |

In [2]:
def get_inner_floor_spec():
    
    return {
        'layers' : [
            {
                'name'                            : 'plywood',
                'thermal_resistance_input_method' : 'conductivity',
                'thermal_conductivity'            : 0.16,
                'volumetric_specific_heat'        : 720.0,
                'thickness'                       : 0.024,
            }
        ]
    }

#### 2) 方位が下面である外皮の部位の面積の合計

$$A_{evlp,down,MR} = \sum_i{A_{evlp,down,MR,i}}$$ 

$$A_{evlp,down,OR} = \sum_i{A_{evlp,down,OR,i}}$$ 

$$A_{evlp,down,NR} = \sum_i{A_{evlp,down,NR,i}}$$ 

$A_{evlp,down,MR}$：主たる居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,OR}$：その他の居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,NR}$：非居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,MR,i}$：主たる居室に接する方位が下面である外皮の部位iの面積, m<sup>2</sup>  
$A_{evlp,down,OR,i}$：その他の居室に接する方位が下面である外皮の部位iの面積, m<sup>2</sup>  
$A_{evlp,down,NR,i}$：非居室に接する方位が下面である外皮の部位iの面積, m<sup>2</sup>  

In [3]:
def get_downward_envelope_total_area(envelope):

    parts = []

    if 'general_parts' in envelope: parts.extend(envelope['general_parts']) 
    
    if 'windows' in envelope: parts.extend(envelope['windows'])
        
    if 'doors' in envelope: parts.extend(envelope['doors'])
    
    def f(space_type):
        return sum(part['area'] for part in parts
                   if part['space_type'] == space_type and (part['direction'] == 'bottom' or part['direction'] == 'downward'))

    return f('main_occupant_room'), f('other_occupant_room'), f('non_occupant_room')

#### 3) 土間床等の中心部の面積の合計

$$A_{ef,MR} = \sum_i {A_{ef,MR,i}}$$ 

$$A_{ef,OR}=\sum_i{A_{ef,OR,i}}$$

$$A_{ef,NR}=\sum_i{A_{ef,NR,i}}$$ 

$$A_{ef,uf}=\sum_i{A_{ef,uf,i}}$$ 

$A_{ef,MR}$：主たる居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,OR}$：その他の居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,NR}$：非居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,uf}$：床下空間に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,MR,i}$：主たる居室に接する土間床等の中心部$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$：その他の居室に接する土間床等の中心部$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$：非居室に接する土間床等の中心部$i$の面積, m<sup>2</sup>  
$A_{ef,uf}$：床下空間に接する土間床等の中心部$i$の面積, m<sup>2</sup>  

In [4]:
def get_earthfloor_total_area(envelope):

    parts = []
    
    if 'earthfloor_centers' in envelope: parts.extend(envelope['earthfloor_centers'])
    
    def f(space_type):
        return sum(part['area'] for part in parts
                   if part['space_type'] == space_type)

    return f('main_occupant_room'), f('other_occupant_room'), f('non_occupant_room'), f('under_floor')

#### 4) 間仕切り床の面積の合計

$$
A_{if,MR} = \max \left( 0, A_{MR} - A_{evlp,down,MR} - A_{ef,MR} \right)
$$

$$
A_{if,OR} = \max \left( 0, A_{OR} - A_{evlp,down,OR} - A_{ef,OR} \right)
$$

$$
A_{if,NR} = \max \left( 0, A_A - A_{MR} - A_{OR} - A_{evlp,down,NR} - A_{ef,NR} \right)
$$

$A_{if,MR}$：床上側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR}$：床上側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR}$：床上側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{A}$: 床面積の合計, m<sup>2</sup>   
$A_{evlp,down,MR}$：主たる居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,OR}$：その他の居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,NR}$：非居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{ef,MR}$：主たる居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,OR}$：その他の居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,NR}$：非居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  

In [5]:
def get_inner_floor_total_area(a_a, a_mr, a_or,
                               a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr,
                               a_ef_mr, a_ef_or, a_ef_nr):

    a_if_mr = max(0.0, a_mr - a_evlp_down_mr - a_ef_mr)
    a_if_or = max(0.0, a_or - a_evlp_down_or - a_ef_or)
    a_if_nr = max(0.0, a_a - a_mr - a_or - a_evlp_down_nr - a_ef_nr)
    
    return a_if_mr, a_if_or, a_if_nr

#### 5) 間仕切り床の面積の合計（床下側が床下空間）

$A_{if,MR} + A_{if,OR} + A_{if,NR} > 0$の場合

$$A_{if,MR-uf} = A_{ef,uf} \frac{A_{if,MR}}{A_{if,MR} + A_{if,OR} + A_{if,NR}}$$

$$A_{if,OR-uf} = A_{ef,uf} \frac{A_{if,OR}}{A_{if,MR} + A_{if,OR} + A_{if,NR}}$$

$$A_{if,NR-uf} = A_{ef,uf} \frac{A_{if,NR}}{A_{if,MR} + A_{if,OR} + A_{if,NR}}$$

$A_{if,MR} + A_{if,OR} + A_{if,NR} = 0$の場合

$$A_{if,MR-uf}=0$$

$$A_{if,OR-uf}=0$$

$$A_{if,NR-uf}=0$$

$A_{if,MR-uf}$：床上側が主たる居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-uf}$：床上側がその他の居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-uf}$：床上側が非居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{ef,uf}$：床下空間に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{if,MR}$：床上側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR}$：床上側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR}$：床上側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  

In [6]:
def get_inner_floor_over_underfloor_total_area(a_if_mr, a_if_or, a_if_nr, a_ef_uf):
    
    if (a_if_mr + a_if_or + a_if_nr) > 0.0:
        a_if_mr_uf = a_ef_uf * a_if_mr / (a_if_mr + a_if_or + a_if_nr)
        a_if_or_uf = a_ef_uf * a_if_or / (a_if_mr + a_if_or + a_if_nr)
        a_if_nr_uf = a_ef_uf * a_if_nr / (a_if_mr + a_if_or + a_if_nr)
    else :
        a_if_mr_uf = 0.0
        a_if_or_uf = 0.0
        a_if_nr_uf = 0.0
    
    return a_if_mr_uf, a_if_or_uf, a_if_nr_uf

#### 6) 間仕切り床の面積の合計（床下側が主たる居室・その他の居室・非居室）

$$ A_{if,MR-OR,NR} = \max{ \left( 0, A_{if,MR}-A_{if,MR-uf} \right) } $$

$$ A_{if,OR-MR,NR} = \max{ \left( 0, A_{if,OR}-A_{if,OR-uf} \right) } $$

$$ A_{if,NR-MR,OR} = \max{ \left( 0, A_{if,NR}-A_{if,NR-uf} \right) } $$

$A_{OR}=0$かつ$A_{NR}=0$の場合

$$A_{if,MR-OR}=0$$

$$A_{if,MR-NR}=0$$

$$A_{if,OR-MR}=0$$

$$A_{if,OR-NR}=0$$

$$A_{if,NR-MR}=0$$

$$A_{if,NR-OR}=0$$

$A_{OR}=0$かつ$A_{NR}\ne0$の場合

$$A_{if,MR-OR}=0$$

$$A_{if,MR-NR}=A_{if,MR-OR,NR}$$

$$A_{if,OR-MR}=0$$

$$A_{if,OR-NR}=0$$

$$A_{if,NR-MR}=A_{if,NR-MR,OR}$$

$$A_{if,NR-OR}=0$$

$A_{OR}\ne0$かつ$A_{NR}=0$の場合

$$A_{if,MR-OR}=A_{if,MR-OR,NR}$$

$$A_{if,MR-NR}=0$$

$$A_{if,NR-MR}=0$$

$$A_{if,NR-OR}=0$$

$$A_{if,OR-MR}=A_{if,OR-MR,NR}$$

$$A_{if,OR-NR}=0$$

$A_{OR}\ne0$かつ$A_{NR}\ne0$の場合

$$A_{if,MR-OR} = \frac{ A_{if,MR-OR,NR} }{ 2 }$$

$$A_{if,MR-NR} = \frac{ A_{if,MR-OR,NR} }{ 2 }$$

$$A_{if,OR-MR} = \frac{ A_{if,OR-MR,NR} }{ 2 }$$

$$A_{if,OR-NR} = \frac{ A_{if,OR-MR,NR} }{ 2 }$$

$$A_{if,NR-MR} = \frac{ A_{if,NR-MR,OR} }{ 2 }$$

$$A_{if,NR-OR} = \frac{ A_{if,NR-MR,OR} }{ 2 }$$

$A_{if,MR-OR,NR}$:床上側が主たる居室に接し床下側がその他の居室又は非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-MR,NR}$:床上側がその他の居室に接し床下側が主たる居室又は非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-MR,OR}$:床上側が非居室に接し床下側が主たる居室又はその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR}$:床上側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR}$:床上側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR}$:床上側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR-uf}$:床上側が主たる居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-uf}$:床上側がその他の居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-uf}$:床上側が非居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR-OR}$:床上側が主たる居室に接し床下側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR-NR}$:床上側が主たる居室に接し床下側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-MR}$:床上側がその他の居室に接し床下側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-NR}$:床上側がその他の居室に接し床下側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-MR}$:床上側が非居室に接し床下側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-OR}$:床上側が非居室に接し床下側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  

In [7]:
def get_inner_floor_between_rooms(a_mr, a_or, a_a, a_if_mr, a_if_or, a_if_nr, a_if_mr_uf, a_if_or_uf, a_if_nr_uf):
    
    a_nr = a_a - a_mr - a_or
    
    if a_mr <= 0.0:
        raise ValueError('主たる居室の面積に0又は負の値が指定されました。主たる居室は必ず0より大きい値を指定する必要があります。')
    
    a_if_mr_or_nr = max(0.0, a_if_mr - a_if_mr_uf)
    a_if_or_mr_nr = max(0.0, a_if_or - a_if_or_uf)
    a_if_nr_mr_or = max(0.0, a_if_nr - a_if_nr_uf)

    if a_or <= 0.0 and a_nr <= 0.0:
        a_if_mr_or = 0.0
        a_if_mr_nr = 0.0
        a_if_or_mr = 0.0
        a_if_or_nr = 0.0
        a_if_nr_mr = 0.0
        a_if_nr_or = 0.0
    elif a_or <= 0.0:
        a_if_mr_or = 0.0
        a_if_mr_nr = a_if_mr_or_nr
        a_if_or_mr = 0.0
        a_if_or_nr = 0.0
        a_if_nr_mr = a_if_nr_mr_or
        a_if_nr_or = 0.0
    elif a_nr <= 0.0:
        a_if_mr_or = a_if_mr_or_nr
        a_if_mr_nr = 0.0
        a_if_or_mr = a_if_or_mr_nr
        a_if_or_nr = 0.0
        a_if_nr_mr = 0.0
        a_if_nr_or = 0.0
    else:
        a_if_mr_or = a_if_mr_or_nr / 2
        a_if_mr_nr = a_if_mr_or_nr / 2
        a_if_or_mr = a_if_or_mr_nr / 2
        a_if_or_nr = a_if_or_mr_nr / 2
        a_if_nr_mr = a_if_nr_mr_or / 2
        a_if_nr_or = a_if_nr_mr_or / 2
    
    return a_if_mr_or, a_if_mr_nr, a_if_or_mr, a_if_or_nr, a_if_nr_mr, a_if_nr_or

#### 3) 室と床下空間の間の床の構築

In [8]:
def get_area_temporary(d):
    
    a_mr = d['common']['main_occupant_room_floor_area']
    a_or = d['common']['other_occupant_room_floor_area']
    a_a  = d['common']['total_floor_area']

    a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr = get_downward_envelope_total_area(d)
    
    a_ef_mr, a_ef_or, a_ef_nr, a_ef_uf = get_earthfloor_total_area(d)

    a_if_mr, a_if_or, a_if_nr = get_inner_floor_total_area(a_a, a_mr, a_or,
                                                           a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr,
                                                           a_ef_mr, a_ef_or, a_ef_nr)

    return get_inner_floor_over_underfloor_total_area(a_if_mr, a_if_or, a_if_nr, a_ef_uf)

In [9]:
def get_floor_area_over_room(d):

    a_mr = d['common']['main_occupant_room_floor_area']
    a_or = d['common']['other_occupant_room_floor_area']
    a_a  = d['common']['total_floor_area']

    a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr = get_downward_envelope_total_area(d)
    
    a_ef_mr, a_ef_or, a_ef_nr, a_ef_uf = get_earthfloor_total_area(d)

    a_if_mr, a_if_or, a_if_nr = get_inner_floor_total_area(a_a, a_mr, a_or,
                                                           a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr,
                                                           a_ef_mr, a_ef_or, a_ef_nr)
    
    a_if_mr_uf, a_if_or_uf, a_if_nr_uf = get_inner_floor_over_underfloor_total_area(a_if_mr, a_if_or, a_if_nr, a_ef_uf)
    
        
    return get_inner_floor_between_rooms(a_mr, a_or, a_a, a_if_mr, a_if_or, a_if_nr, a_if_mr_uf, a_if_or_uf, a_if_nr_uf)

In [10]:
def get_innerwall_groundfloor(d):
    
    # 室と床下空間の間の床面積の推定
    a_if_mr, a_if_or, a_if_nr = get_area_temporary(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    ifs = []
    
    if a_if_mr > 0.0:
        ifs.append({
            'name' : 'GroundFloor_Main',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_mr,
            'Layers' : get_inner_floor_spec(),
            'space_type' : 'main',
            'nextspace': 'underfloor'
        })
        ifs.append({
            'name' : 'GroundFloor_Main',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_mr,
            'Layers' : get_inner_floor_spec(),
            'space_type' : 'underfloor',
            'nextspace': 'main'
        })
    
    if a_if_or > 0.0:
        ifs.append({
            'name' : 'GroundFloor_Other',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_or,
            'Layers' : get_inner_floor_spec(),
            'space_type' : 'other',
            'nextspace': 'underfloor'
        })
        ifs.append({
            'name' : 'GroundFloor_Other',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_or,
            'Layers' : get_inner_floor_spec(),
            'space_type' : 'underfloor',
            'nextspace': 'other'                
        })
        
    if a_if_nr > 0.0:
        ifs.append({
            'name' : 'GroundFloor_Nonliving',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_nr,
            'Layers' : get_inner_floor_spec(),
            'space_type' : 'nonliving',
            'nextspace': 'underfloor'
        })
        ifs.append({
            'name' : 'GroundFloor_Nonliving',
            'type' : 'GroundFloor',
            'direction' : 'Horizontal',
            'area' : a_if_nr,
            'Layers' : get_inner_floor_spec(),
            'space_type' : 'underfloor',
            'nextspace': 'nonliving'
        })
        
    return ifs

##### Example

In [11]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space_type': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space_type': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space_type': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'earthfloor_centers': [
        { 'name': 'other', 'area': 5.0, 'space_type': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
    ]
}

innerwall_groundfloor = get_innerwall_groundfloor(d)
innerwall_groundfloor

[]

### 8.3. InnerWalls Element - Inner Floor

#### 2) 室上の床面積の推定

##### Function

##### Example

In [12]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space_type': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space_type': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space_type': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'earthfloor_centers': [
        { 'name': 'other', 'area': 5.0, 'space_type': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'space_type': 'main', 'nextspace' : 'underfloor', 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'space_type': 'other', 'nextspace' : 'underfloor', 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
]

get_floor_area_over_room(d)

(15.0, 15.0, 30.0, 30.0, 15.0, 15.0)

#### 3) 室間の床と天井の面積の推定

##### Function 

In [13]:
def get_slab_area_between_rooms(d, innerwalls):
    
    def get_if(space_type_name, next_space_name, area):
        return {
            'name' : 'InnerFloor' + space_type_name + next_space_name,
            'area' : area,
            'space_type' : space_type_name,
            'nextspace' : next_space_name
        }
    
    def get_ic(space_type_name, next_space_name, area):
        return {
            'name' : 'InnerCeiling' + space_type_name + next_space_name,
            'area' : area,
            'space_type' : space_type_name,
            'nextspace' : next_space_name
        }
    
    # 室上の床面積の取得
    a_if_mr_or, a_if_mr_nr, a_if_or_mr, a_if_or_nr, a_if_nr_mr, a_if_nr_or = get_floor_area_over_room(d)

    # 部屋間の床面積・天井面積を求める。
    ifs = []
    ics = []
    
    if a_if_mr > 0.0 and a_if_or > 0.0 and a_if_nr > 0.0:

        ifs.append(get_if('main',      'other',     a_if_mr_or))
        ifs.append(get_if('main',      'nonliving', a_if_mr_nr))
        ifs.append(get_if('other',     'main',      a_if_or_mr))
        ifs.append(get_if('other',     'nonliving', a_if_or_nr))
        ifs.append(get_if('nonliving', 'main',      a_if_nr_mr))
        ifs.append(get_if('nonliving', 'other',     a_if_nr_or))

        ics.append(get_ic('other',     'main',      a_if_mr_or))
        ics.append(get_ic('main',      'other',     a_if_or_mr))
        ics.append(get_ic('nonliving', 'main',      a_if_mr_nr))
        ics.append(get_ic('main',      'nonliving', a_if_nr_mr))
        ics.append(get_ic('nonliving', 'other',     a_if_or_nr))
        ics.append(get_ic('other',     'nonliving', a_if_nr_or))
        
    elif a_if_mr > 0.0 and a_if_or > 0.0:

        ifs.append(get_if('main', 'other', a_if_mr))
        ifs.append(get_if('other', 'main', a_if_or))
 
        ics.append(get_ic('other', 'main', a_if_mr))
        ics.append(get_ic('main', 'other', a_if_or))      
         
    elif a_if_mr > 0.0 and a_if_nr > 0.0:
        
        ifs.append(get_if('main', 'nonliving', a_if_mr))
        ifs.append(get_if('nonliving', 'main', a_if_nr))

        ics.append(get_ic('nonliving', 'main', a_if_mr))
        ics.append(get_ic('main', 'nonliving', a_if_nr))
        
    elif a_if_or > 0.0 and a_if_nr > 0.0:
        
        ifs.append(get_if('other', 'nonliving', a_if_or))
        ifs.append(get_if('nonliving', 'other', a_if_nr))

        ics.append(get_ic('nonliving', 'other', a_if_or))
        ics.append(get_ic('other', 'nonliving', a_if_nr))
  
    return {'InnerFloors' : ifs, 'InnerCeilings' : ics}

#### 4) 室間の床・天井の構築

##### Function

In [14]:
def get_innerwall_innerslab(d, innerwalls):
    
    # 室間の床・天井面積の推定
    areas_innerslab = get_slab_area_between_rooms(d, innerwalls)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    ifs = []
    
    ifs.extend([ {
        'name' : 'InnerFloor_' + x['space_type'],
        'type' : 'InnerFloor',
        'direction' : 'Horizontal',
        'area' : x['area'],
        'Layers' : get_inner_floor_spec(),
        'space_type' : x['space_type'], 
        'nextspace' : x['nextspace'],
    } for x in areas_innerslab['InnerFloors'] ])
    
    ifs.extend([ {
        'name' : 'InnerCeiling_' + x['space_type'],
        'type' : 'InnerCeiling',
        'direction' : 'Horizontal',
        'area' : x['area'],
        'Layers' : get_inner_floor_spec(),
        'space_type' : x['space_type'], 
        'nextspace' : x['nextspace'],
    } for x in areas_innerslab['InnerCeilings'] ])
        
    return ifs

#### Example

In [15]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space_type': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space_type': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space_type': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'earthfloor_centers': [
        { 'name': 'other', 'area': 5.0, 'space_type': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'space_type': 'main', 'nextspace' : 'underfloor', 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'space_type': 'other', 'nextspace' : 'underfloor', 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
]

get_slab_area_between_rooms(d, innerwalls)

NameError: name 'a_if_mr' is not defined

### 8.5. InnerWalls Element - Inner Wall 

#### 4) 方位が水平（上面・下面以外）である外皮の部位の面積の合計

In [ ]:
def get_outside_wall_area(d):

    parts = []

    if 'general_parts' in d:
        parts.extend(d['general_parts'])
        
    if 'windows' in d:
        parts.extend(d['windows'])
        
    if 'doors' in d:
        parts.extend(d['doors'])

    def f(room_type):
        return sum(part['area'] for part in parts \
                   if part['space_type'] == room_type and part['direction'] != 'top' and part['direction'] != 'bottom')
    
    return f('Main'), f('Other'), f('Nonliving')

#### 1) 層構成

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup>2 k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |
| 非密閉空気層 | - | - | 0.090 | 0.0 |
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |

##### Function

In [ ]:
def get_inner_wall_layers():
    return [
        {'name': 'GPB', 'cond': 0.22,   'thick': 0.0125, 'specH': 830.0},
        {'name': 'GPB', 'cond': 1/0.09, 'thick': 1,      'specH': 0},
        {'name': 'GPB', 'cond': 0.22,   'thick': 0.0125, 'specH': 830.0}
    ]

#### 2) 内壁等面積の推定

用途間の内壁等の面積は、各用途の内壁等の面積より決定する。

$$
A_{iw,MR-OR} =\frac{ A_{iw,MR} + A_{iw,OR} - A_{iw,NR} }{2}
$$

$$
A_{iw,MR-NR} = \frac{ A_{iw,MR} + A_{iw,NR} - A_{iw,OR} }{2}
$$

$$
A_{iw,NR-OR} = \frac{ A_{iw,NR} + A_{iw,OR} - A_{iw,MR} }{2}
$$

$ \because $

$$
A_{iw,MR} =A_{iw,MR-OR}+A_{iw,MR-NR}
$$

$$
A_{iw,OR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$$
A_{iw,NR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$A_{iw,MR-OR}$: 主たる居室とその他の居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR-NR}$: 主たる居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR-NR}$: その他の居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR}$: 主たる居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR}$: その他の居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,NR}$: 非居室の内壁等の面積, m<sup>2</sup>  

各用途の内壁等の面積は、各用途の内壁等の面積の暫定値より、用途間の内壁等の面積が0㎡以上となる範囲で決定する。

$A_{iw,MR,temp}>A_{iw,OR,temp}+A_{iw,NR,temp}$ の場合

　　$A_{OR}=0$ かつ $A_{NR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,MR} = A_{iw,MR,temp}
$$
$$
A_{iw,OR} = A_{iw,MR,temp}\times \frac{A_{OR}}{A_{OR}+A_{NR}}
$$
$$
A_{iw,NR} = A_{iw,MR,temp}\times \frac{A_{NR}}{A_{OR}+A_{NR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,MR} = A_{iw,MR,temp}
$$
$$
A_{iw,OR} = A_{iw,MR,temp}\times \frac{A_{iw,OR,temp}}{A_{iw,OR,temp}+A_{iw,NR,temp}}
$$
$$
A_{iw,NR} = A_{iw,MR,temp}\times \frac{A_{iw,NR,temp}}{A_{iw,OR,temp}+A_{iw,NR,temp}}
$$

$A_{iw,OR,temp}>A_{iw,MR,temp}+A_{iw,NR,temp}$ の場合

　　$A_{MR}=0$ かつ $A_{NR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,OR} = A_{iw,OR,temp}
$$
$$
A_{iw,MR} = A_{iw,OR,temp}\times \frac{A_{MR}}{A_{MR}+A_{NR}}
$$
$$
A_{iw,NR} = A_{iw,OR,temp}\times \frac{A_{NR}}{A_{MR}+A_{NR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,OR} = A_{iw,OR,temp}
$$
$$
A_{iw,MR} = A_{iw,OR,temp}\times \frac{A_{iw,MR,temp}}{A_{iw,MR,temp}+A_{iw,NR,temp}}
$$
$$
A_{iw,NR} = A_{iw,OR,temp}\times \frac{A_{iw,NR,temp}}{A_{iw,MR,temp}+A_{iw,NR,temp}}
$$

$A_{iw,NR,temp}>A_{iw,MR,temp}+A_{iw,OR,temp}$ の場合

　　$A_{MR}=0$ かつ $A_{OR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,NR} = A_{iw,NR,temp}
$$
$$
A_{iw,MR} = A_{iw,NR,temp}\times \frac{A_{MR}}{A_{MR}+A_{OR}}
$$
$$
A_{iw,OR} = A_{iw,NR,temp}\times \frac{A_{OR}}{A_{MR}+A_{OR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,NR} = A_{iw,NR,temp}
$$
$$
A_{iw,MR} = A_{iw,NR,temp}\times \frac{A_{iw,MR,temp}}{A_{iw,MR,temp}+A_{iw,OR,temp}}
$$
$$
A_{iw,OR} = A_{iw,NR,temp}\times \frac{A_{iw,OR,temp}}{A_{iw,MR,temp}+A_{iw,OR,temp}}
$$

上記以外の場合

$$A_{iw,MR} = A_{iw,MR,temp}, A_{iw,OR} = A_{iw,OR,temp}, A_{iw,NR} = A_{iw,NR,temp}$$

$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{iw,MR,temp}$: 主たる居室の内壁等の面積の暫定値, m<sup>2</sup>  
$A_{iw,OR,temp}$: その他の居室の内壁等の面積の暫定値, m<sup>2</sup>  
$A_{iw,NR,temp}$: 非居室の内壁等の面積の暫定値, m<sup>2</sup>  

各用途の内壁等の面積の暫定値は、室の周長を求める際の係数（アスペクト比）と階高を仮定して、外皮の用途別情報および用途別床面積より推定する。

$$
A_{iw,MR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{MR} = 0 ) \\
A_{w,MR} - A_{ow,MR} = \max \left ( 0,4 \cdot ar_{MR} \cdot h \sqrt {A_{MR}} - A_{ow,MR} \right ) & ( A_{MR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,OR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{OR} = 0 ) \\
A_{w,OR} - A_{ow,OR} = \max \left ( 0, 4 \cdot ar_{NR} \cdot h \sqrt {A_{OR}} - A_{ow,OR} \right ) & ( A_{OR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,NR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{NR} = 0 ) \\
A_{w,NR} - A_{ow,NR} = \max \left ( 0, 4 \cdot ar_{OR} \cdot h \sqrt {A_{NR}} - A_{ow,NR} \right ) & ( A_{NR} > 0 )
\end{array}
\right.
$$

$A_{w,MR}$: 主たる居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{w,OR}$: その他の居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{w,NR}$: 非居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{ow,MR}$: 主たる居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$A_{ow,OR}$: その他の居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$A_{ow,NR}$: 非居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$ar_{MR}$: 主たる居室の周長を求める際の係数, -  
$ar_{OR}$: その他の居室の周長を求める際の係数, -  
$ar_{NR}$: 非居室の周長を求める際の係数, -  
$h$: 階高, m

In [ ]:
def get_floor_area(a_mr, a_or, a_total):
    return a_mr, a_or, a_total - a_mr - a_or

##### Function

In [ ]:
def get_area_inner_wall(d):
    
    # 用途別床面積の取得
    a_mr, a_or, a_nr = get_floor_area(a_mr=d['common']['main_occupant_room_floor_area'],
                                      a_or=d['common']['other_occupant_room_floor_area'],
                                      a_total=d['common']['total_floor_area'])
 
    # 周長を求める際の係数, -
    r_mr, r_or, r_nr = 1.2, 1.4, 1.4

    # 居室の外皮と床下空間との間の床の面積の合計
    a_ow_mr, a_ow_or, a_ow_nr = get_outside_wall_area(d)

    # 内壁等の面積の暫定値の取得
    def get_int_wall_area(floor_area, ratio, outside_wall_area):
        h = 2.8 # 階高, m
        return max(0.0, 4 * h * ratio * floor_area**0.5 - outside_wall_area) if floor_area > 0.0 else 0.0

    # 内壁等の面積の暫定値の取得
    a_iw_mr = get_int_wall_area(a_mr, r_mr, a_ow_mr)
    a_iw_or = get_int_wall_area(a_or, r_or, a_ow_or)
    a_iw_nr = get_int_wall_area(a_nr, r_nr, a_ow_nr)

    # 内壁等の面積の取得
    def get_alt_area(a_iw, a_iw_adj1, a_iw_adj2, a_f_adj1, a_f_adj2):
        # a_iw: 当該ZONEの内壁面積
        # a_iw_adj1, a_f_adj1: 隣のZONE1の内壁面積, 床面積
        # a_iw_adj2, a_f_adj2: 隣のZONE2の内壁面積, 床面積
        
        if a_f_adj1 == 0.0 and a_f_adj2 == 0.0:
            return 0.0, 0.0, 0.0
        elif a_iw_adj1 == 0.0 and a_iw_adj2 == 0.0:
            return a_iw, a_iw * a_f_adj1 / (a_f_adj1 + a_f_adj2), a_iw * a_f_adj2 / (a_f_adj1 + a_f_adj2)
        else:
            return a_iw, a_iw * a_iw_adj1 / (a_iw_adj1 + a_iw_adj2), a_iw * a_iw_adj2 / (a_iw_adj1 + a_iw_adj2)

    if a_iw_mr > a_iw_or + a_iw_nr:
        alt_a_iw_mr, alt_a_iw_or, alt_a_iw_nr = get_alt_area(a_iw_mr, a_iw_or, a_iw_nr, a_or, a_nr)
    elif a_iw_or > a_iw_nr + a_iw_mr:
        alt_a_iw_or, alt_a_iw_nr, alt_a_iw_mr = get_alt_area(a_iw_or, a_iw_nr, a_iw_mr, a_nr, a_mr)
    elif a_iw_nr > a_iw_mr + a_iw_or:
        alt_a_iw_nr, alt_a_iw_mr, alt_a_iw_or = get_alt_area(a_iw_nr, a_iw_mr, a_iw_or, a_mr, a_or)
    else :
        alt_a_iw_mr, alt_a_iw_or, alt_a_iw_nr = a_iw_mr, a_iw_or, a_iw_nr
    
    # 室用途間の内壁等の面積の推定
    a_iw_mr_or = (alt_a_iw_mr + alt_a_iw_or - alt_a_iw_nr) / 2
    a_iw_mr_nr = (alt_a_iw_nr + alt_a_iw_mr - alt_a_iw_or) / 2
    a_iw_or_nr = (alt_a_iw_or + alt_a_iw_nr - alt_a_iw_mr) / 2
    
    return a_iw_mr_or, a_iw_mr_nr, a_iw_or_nr           

#### 3) 内壁の構築

##### Function

In [ ]:
def get_innerwall_wall(d):

    # 内壁面積の推定
    a_iw_mr_or, a_iw_mr_nr, a_iw_or_nr = get_area_inner_wall(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    iws = []

    if a_iw_mr_or > 0.0:

        iws.append({
            'name' : 'Innerwall_main to other',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_mr_or,
            'space_type' : 'main', 
            'nextspace': 'other'
        })
        
        iws.append({
            'name' : 'Innerwall_main to other',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_mr_or,
            'space_type' : 'other', 
            'nextspace': 'main'
        })

    if a_iw_mr_nr > 0.0:
        
        iws.append({
            'name' : 'Innerwall_main to nonliving',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_mr_nr,
            'space_type' : 'main', 
            'nextspace': 'nonliving'
        })
        
        iws.append({
            'name' : 'Innerwall_main to nonliving',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_mr_nr,
            'space_type' : 'nonliving', 
            'nextspace': 'main'
        })

    if a_iw_or_nr > 0.0:
        
        iws.append({
            'name' : 'Innerwall_other to nonliving',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_or_nr,
            'space_type' : 'other', 
            'nextspace': 'nonliving'
        })
        iws.append({
            'name' : 'Innerwall_other to nonliving',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : a_iw_or_nr,
            'space_type' : 'nonliving', 
            'nextspace': 'other'
        })

    return iws

##### Example

In [ ]:
correspondence = {
    'main to other': ['main', 'other'],
    'main to nonliving': ['main', 'nonliving'],
    'other to nonliving': ['other', 'nonliving'] 
}  

d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space_type': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space_type': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space_type': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'earthfloor_centers': [
        { 'name': 'other', 'area': 5.0, 'space_type': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
]

get_innerwall_wall(d)

### 8.3. InnerWalls Element - 統合

##### Function

In [ ]:
def convert_innerwall(d):

    # 室と床下空間の間の床の推定
    iws_to_groundfloor = get_innerwall_groundfloor(d)
    
    # 室間の床・天井の推定
    iws_slab = get_innerwall_innerslab(d, iws_to_groundfloor)
    
    # 内壁等の推定              
    iws_wall = get_innerwall_wall(d)
    
    return iws_to_groundfloor + iws_slab + iws_wall

##### Example

In [ ]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space_type': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space_type': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space_type': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'earthfloor_centers': [
        { 'name': 'other', 'area': 5.0, 'space_type': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
]

convert_innerwall( d )

## 9. Lv4入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床、8)内壁等の各コンバート関数を統合し、Lv3の入力情報をLv4の入力情報にコンバートする。

##### Function

In [ ]:
def convert(d):
    
    d_lv4_input = {}
    
    d_lv4_input['common'] = copy.deepcopy(d['common'])
    
    if ('general_parts' in d) == True:
        d_lv4_input['general_parts'] = copy.deepcopy(d['general_parts'])
        
    if ('windows' in d) == True:
        d_lv4_input['windows'] = copy.deepcopy(d['windows'])
        
    if ('doors' in d) == True:
        d_lv4_input['doors'] = copy.deepcopy(d['doors'])
        
    if ('heatbridges' in d) == True:
        d_lv4_input['heatbridges'] = copy.deepcopy(d['heatbridges'])
        
    if ('earthfloor_perimeters' in d) == True:
        d_lv4_input['earthfloor_perimeters'] = copy.deepcopy(d['earthfloor_perimeters'])
        
    if ('earthfloor_centers' in d) == True:
        d_lv4_input['earthfloor_centers'] = copy.deepcopy(d['earthfloor_centers'])
        
    d_lv4_input['InnerWalls'] = convert_innerwall(d)

    return d_lv4_input

##### Example

In [ ]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 67.8, 'space_type': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space_type': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'space_type': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 67.8,
          'ConstructionMethodAllLayers' :'FloorFrameBeamInsuljoist',
          'Parts': [{'Type': 'FloorInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'FloorHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False },
        { 'name': 'Wall', 'type': 'Wall', 'space_type': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'N',
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'WallFrameInsulcolumn',
          'Parts': [{'Type': 'WallInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'WallHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Roof', 'space_type': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'top',
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'RoofInsulRafter', 
          'Parts': [{'Type': 'RoofInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'RoofHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'BoundaryCeiling', 'type': 'BoundaryCeiling', 'space_type': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'CeilingInsulbeam', 
          'Parts': [{'Type': 'CeilingInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'CeilingHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUR', 'InsulationMethod': 'CeilingInsulbeam',
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space_type': 'main', 
          'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                 {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod' :'InputLayers', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space_type': 'main',
          'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                 {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod' :'InputUR', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space_type': 'main', 'Radding': 0.10, 
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'other', 'direction': 'top', 'IsInContactWithOutsideAir': True, 
          'area': 67.8, 'space_type': 'main', 'IsSunshadeInput': False }
    ],
    'windows': [
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'EtaInputMethod': 'InputSpecification', 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'space_type': 'main', 'UW': 4.65, 'TypeWindow': 'Single', 
          'EtaInputMethod': 'InputValue', 'Eta': 0.738, 'IsSunshadeInput': False },
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Double', 
          'EtaInputMethod': 'InputSpecification', 'TypeFrameInside': 'WoodOrResin', 'TypeGlassInside': '3WgG', 'TypeShadeInside': 'Shoji',
          'TypeFrameOutside': 'Steel', 'TypeGlassOutside': '3WgG', 'TypeShadeOutside': 'ExtarnalBlind',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'space_type': 'main', 'UW': 4.65, 'TypeWindow': 'Double',
          'EtaInputMethod': 'InputSpecification', 'EtaInside': 0.738, 'TypeGlassInside': '3WgG', 'EtaOutside': 0.738, 
          'TypeGlassOutside': '3WgG', 'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'doors': [
        { 'name': 'DoorNW', 'direction': 'NW', 'area': 2.52, 'U': 6.51, 'space_type': 'main', 'IsSunshadeInput': False },
        { 'name': 'DoorNE', 'direction': 'NE', 'area': 2.16, 'U': 4.65, 'space_type': 'main', 
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'heatbridges': [
        { 'name': 'NE', 'structure': 'RC', 'length': 1.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'E', 'space_type': 'main' },
        { 'name': 'NW', 'structure': 'Steel', 'length': 2.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'W', 'space_type': 'main' }
    ],
    'earthfloor_perimeters': [
        { 'name': 'NW', 'direction': 'NW', 'length': 2.43, 'psi': 1.8, 'space_type': 'underfloor' },
        { 'name': 'NE', 'direction': 'NE', 'length': 2.43, 'psi': 1.8, 'space_type': 'underfloor' }
    ],
    'earthfloor_centers': [
        { 'name': 'other', 'area': 5.0, 'space_type': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
    ]
}

convert(d)

In [ ]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 30.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        {'name': 'Ceiling_main', 'direction': 'Top', 'area': 16.95, 'space_type': 'main', 'type': 'Ceiling',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Ceiling_other', 'direction': 'Top', 'area': 16.95, 'space_type': 'other', 'type': 'Ceiling',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Ceiling_nonliving', 'direction': 'Top', 'area': 33.9, 'space_type': 'nonliving', 'type': 'Ceiling',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Wall_SW_main', 'direction': 'SW', 'area': 10.1575, 'space_type': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SW_other', 'direction': 'SW', 'area': 10.1575, 'space_type': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SW_nonliving', 'direction': 'SW', 'area': 20.315, 'space_type': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_main', 'direction': 'NW', 'area': 7.4575, 'space_type': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_other', 'direction': 'NW', 'area': 7.4575, 'space_type': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_nonliving', 'direction': 'NW', 'area': 14.915, 'space_type': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_main', 'direction': 'NE', 'area': 15.9725, 'space_type': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_other', 'direction': 'NE', 'area': 15.9725, 'space_type': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_nonliving', 'direction': 'NE', 'area': 31.945, 'space_type': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_main', 'direction': 'SE', 'area': 7.4275, 'space_type': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_other', 'direction': 'SE', 'area': 7.4275, 'space_type': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_nonliving', 'direction': 'SE', 'area': 14.855, 'space_type': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Floor_main', 'direction': 'Bottom', 'area': 15.0175, 'space_type': 'main', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_other', 'direction': 'Bottom', 'area': 15.0175, 'space_type': 'other', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_nonliving', 'direction': 'Bottom', 'area': 30.035, 'space_type': 'nonliving', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_main', 'direction': 'Bottom', 'area': 1.1025, 'space_type': 'main', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_other', 'direction': 'Bottom', 'area': 1.1025, 'space_type': 'other', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_nonliving', 'direction': 'Bottom', 'area': 2.205, 'space_type': 'nonliving', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27}
    ],
    'windows': [
        {'name': 'WindowSW_main', 'direction': 'SW', 'area': 7.5625, 'space_type': 'main', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSW_other', 'direction': 'SW', 'area': 7.5625, 'space_type': 'other', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSW_nonliving', 'direction': 'SW', 'area': 15.125, 'space_type': 'nonliving', 'UW': 3.49, 
         'EtaInputMethod': 'InputSeasonalValue', 'EtaCooling': 0.51, 'EtaHeating': 0.51,
         'IsSunshadeInput': True, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNW_main', 'direction': 'NW', 'area': 0.7925, 'space_type': 'main', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNW_other', 'direction': 'NW', 'area': 0.7925, 'space_type': 'other', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNW_nonliving', 'direction': 'NW', 'area': 1.585, 'space_type': 'nonliving', 'UW': 3.49, 
         'EtaInputMethod': 'InputSeasonalValue', 'EtaCooling': 0.51, 'EtaHeating': 0.51,
         'IsSunshadeInput': True, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNE_main', 'direction': 'NE', 'area': 1.21, 'space_type': 'main', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNE_other', 'direction': 'NE', 'area': 1.21, 'space_type': 'other', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNE_nonliving', 'direction': 'NE', 'area': 2.42, 'space_type': 'nonliving', 'UW': 3.49, 
         'EtaInputMethod': 'InputSeasonalValue', 'EtaCooling': 0.51, 'EtaHeating': 0.51,
         'IsSunshadeInput': True, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSE_main', 'direction': 'SE', 'area': 1.4575, 'space_type': 'main', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSE_other', 'direction': 'SE', 'area': 1.4575, 'space_type': 'other', 'UW': 3.49, 
         'EtaInputMethod': 'InputSeasonalValue', 'EtaCooling': 0.51, 'EtaHeating': 0.51, 
         'IsSunshadeInput': True, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSE_nonliving', 'direction': 'SE', 'area': 2.915, 'space_type': 'nonliving', 'UW': 3.49,
         'EtaInputMethod': 'InputSeasonalValue', 'EtaCooling': 0.51, 'EtaHeating': 0.51, 
         'IsSunshadeInput': True, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None}
    ],
    'doors': [
        {'name': 'DoorNW_main', 'direction': 'NW', 'area': 0.63, 'space_type': 'main', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNW_other', 'direction': 'NW', 'area': 0.63, 'space_type': 'other', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNW_nonliving', 'direction': 'NW', 'area': 1.26, 'space_type': 'nonliving', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_main', 'direction': 'NE', 'area': 0.54, 'space_type': 'main', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_other', 'direction': 'NE', 'area': 0.54, 'space_type': 'other', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_nonliving', 'direction': 'NE', 'area': 1.08, 'space_type': 'nonliving', 'U': 4.65, 'IsSunshadeInput': False}
    ],
    'earthfloor_perimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'Entrance_NW', 'psi': 1.8, 'space_type': 'underfloor'},
        {'direction': 'NE', 'length': 1.83, 'name': 'Entrance_NE', 'psi': 1.8, 'space_type': 'underfloor'},
        {'direction': 'OpenBackFloor', 'length': 4.25, 'name': 'Entrance_floor', 'psi': 1.8, 'space_type': 'underfloor'}
    ]
}

convert(d)